The purpose of this notebook is to explore our scraped tweets with query of the hashtag "#COVID19". In this we are going to first preprocess the tweets, perform some visualisations and later on investigate if there is a correlation between COVID19 and scarcity and if so, try to quantify it. An issue with this very model is that we require a benchmark to compare against, otherwise the values we report are effectively useless. Maybe a better model would be to query the hashtag "#Hunger" or "#Poverty" and see how many associations there are with coronavirus. Again, past data would come in handly. Perhaps a Premium Twitter Developer account may help and we can form a partnership of some kind with the right people to aid in this quest for analysis of tweets. More visualiastion here than NLP.


Author: Steven Vuong<br>
Last Edited: 11-05-2020

In [ ]:
import pandas as pd

csv_savepath = "../../data/tweets.csv"
tweets_df = pd.read_csv(csv_savepath) # Of query "#COVID19"

In [ ]:
tweets_df.head()

We will try get some ideas for preprocessing from other notebooks/kernels.

-  Idea: Try to predict if tweet contains hashtag "#Hunger" or related. Can use tweet text and other data to try and predict then ensemble. Brilliant template for model training: https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle
-  Preprocessing: https://www.kaggle.com/sudalairajkumar/getting-started-with-text-preprocessing
-  Visualisation: https://www.kaggle.com/duttadebadri/detailed-nlp-project-prediction-visualization
    - Histogram/barplots for frequency
    - Wordcloud (Hoping to do post processing)
    - Word correlation map
    
Also to consider: unigrams/bigrams/trigrams


In [ ]:
from text_preprocessor import TextPreprocessor

In [ ]:
print("-----------Sample Text-----------")
txt_sample = TextPreprocessor(tweets_df["tweet_text"][0])
print(txt_sample.text)

# Apply preprocessing steps
txt_sample._make_lowercase()
txt_sample._remove_punctuation()
txt_sample._remove_stopwords()
txt_sample._lemmatize()
txt_sample._remove_emojis()
txt_sample._remove_emoticons()
txt_sample._remove_urls()
txt_sample._remove_html_tags()
txt_sample._spellcheck()

print("\n----------Post Processed-----------")
print(txt_sample.text)

Preprocessing notes for this use case:

- We opt for lemmatizing in this use case, just that the example output looks better >> Would require more samples to be sure.
- We opt for removing emoticons and emojis than converting to words as we want to have a simple model to begin with and conversion may not be so greatly depicted in tweets. May be for other use cases
- Keeping slang as it may contain important nuances.

In [ ]:
# TODO: Add Stanford NER tagger to TextProcessor Class
# Also considerr stanford NER (named entity recognition) Tagger
# https://towardsdatascience.com/tweet-analytics-using-nlp-f83b9f7f7349
# from nltk.tag import StanfordNERTagger
# from nltk.tokenize import word_tokenize

# st = StanfordNERTagger('/stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz','/usr/share/stanford-ner/stanford-ner.jar',encoding='utf-8')
# #a tweet by Donald Trump
# text = 'Just had a very good call with @SwedishPM Stefan Löfven who assured me that American citizen A$AP Rocky will be treated fairly. Likewise, I assured him that A$AP was not a flight risk and offered to personally vouch for his bail, or an alternative....'

# tokenized_text = word_tokenize(text)
# classified_text = st.tag(tokenized_text)

# print(classified_text)

In [ ]:
# TODO
# Make a plot function -> histogram, worcloud ettc..
    # Or does that belong in a separate func? Maybe clas with static
    # metthods hauah!
# Remove frequent words would have to be performed outside as analysis of 
# all the words first, then remove. Same applies for removing rare words.


In [ ]:
# TODO: Remove most frequent words and most rare words from text
# TODO: Make a loop to process all the words in our dataframe